# Multi Layer Perceptron #

## Lab 05 : Multi Layer Perceptron ##

### Review ###  

**모델(Model)**
$y = h(W_2h(W_1x + b_1) + b_2)$  
-> Model은 Weight Vector W와 bias vector b를 더한 뒤, Activation함수를 거치고, 이를 여려겹 쌓은 것.  

**모수(Parameter)**  
$W_1$, $W_2$, $b_1$, $b_2$ : Vectors of Weights and Bias

**오차 함수(Error Function)**  
Regression : MSE, $L = \frac{1}{N}\sum_{k=1}^N (y - \hat{y})^2$  
Classification : Cross Entropy : $L = -\sum_i \hat{y}_ilog(y_i)$  

**학습 알고리즘(Learning Algorithm)**  
Gradient Descent and Error Back Propagation  
- 출력단의 Error를 앞쪽 Layer로 전파(Propagate)하며, Gradient가 감소하는 방향으로 Weight를 Update

### Load Dataset ###
Multi Layer Perceptron은 기존에 Perceptron 알고리즘으로는 해결할 수 없었던 XOR 분류와 같은 문제를 해결할 수 있도록 해주었습니다.  
마치, Logic Gate를 여러개 사용하여 XOR를 만들어낼 수 있듯이, Perceptron 역시 여러개의 Layer를 사용하면 XOR를 표현할 수 있었습니다.  

그러면, 한번 XOR를 직접 학습해보도록 하겠습니다.

In [ ]:
# 필요한 라이브러리들을 import 해보겠습니다.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import os
import imageio

In [ ]:
# 데이터셋을 만들어볼게요.
x_1, y_1 = np.random.multivariate_normal([2, 2], [[1, 0], [0, 1]], 500).T
x_2, y_2 = np.random.multivariate_normal([-2, 2], [[1, 0], [0, 1]], 500).T
x_3, y_3 = np.random.multivariate_normal([-2, -2], [[1, 0], [0, 1]], 500).T
x_4, y_4 = np.random.multivariate_normal([2, -2], [[1, 0], [0, 1]], 500).T

X = np.concatenate([x_1, x_3, x_2, x_4])
Y = np.concatenate([y_1, y_3, y_2, y_4])
Z = np.concatenate([np.ones(1000), np.zeros(1000)])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

# Move left y-axis and bottim x-axis to centre, passing through (0,0)
ax.spines['left'].set_position('center')
ax.spines['bottom'].set_position('center')

# Eliminate upper and right axes
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')

# Show ticks in the left and lower axes only
ax.xaxis.set_ticks_position('bottom')

plt.scatter(X, Y, c=Z, cmap=plt.cm.Set3)

1사분면과 3사분면엔 노란색의 데이터가 있고, 2사분면과 4사분면엔 민트색의 데이터가 있습니다!  
자, 그러면 이 데이터를 가지고 지난번에 했던 Logistic Regression을 가지고 학습을 진행해볼게요.

In [ ]:
class XORDataset(data.Dataset):
    def __init__(self):
        super(XORDataset, self).__init__()
        
        x_1, y_1 = np.random.multivariate_normal([2, 2], [[1, 0], [0, 1]], 500).T
        x_2, y_2 = np.random.multivariate_normal([-2, 2], [[1, 0], [0, 1]], 500).T
        x_3, y_3 = np.random.multivariate_normal([-2, -2], [[1, 0], [0, 1]], 500).T
        x_4, y_4 = np.random.multivariate_normal([2, -2], [[1, 0], [0, 1]], 500).T

        self.X = np.stack([np.concatenate([x_1, x_3, x_2, x_4]), np.concatenate([y_1, y_3, y_2, y_4])], axis=1)
        self.Y = np.concatenate([np.ones(1000), np.zeros(1000)])

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        data = torch.FloatTensor(self.X[index])
        label = torch.FloatTensor([self.Y[index]])
        return data, label

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.fc1 = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        return x

In [ ]:
model = LinearRegressionModel().to(device)
model(torch.FloatTensor([1, 2]).to(device))

모델을 가지고 학습 전, 현재 이 모델이 어떻게 분류를 하고 있는지 한번 볼게요.

In [ ]:
def visualize(X, Y, Z, model, save_idx=None, device='cuda'):
    x_min, x_max = X.min() - .5, X.max() +.5
    y_min, y_max = Y.min() - .5, Y.max() +.5

    ar_X, ar_Y = np.meshgrid(np.arange(x_min, x_max, .02),
                             np.arange(y_min, y_max, .02))

    input_arange = np.array([ar_X.ravel(), ar_Y.ravel()])
    input_arange = input_arange.T
    Z_hat = model(torch.FloatTensor(input_arange).to(device)).detach().cpu().numpy()
    Z_hat = Z_hat.round()
    Z_hat = Z_hat.reshape(ar_X.shape)
    
    plt.figure(1)
    plt.pcolormesh(ar_X, ar_Y, Z_hat, cmap=plt.cm.Set3)
    plt.scatter(X, Y, c=Z, edgecolors='k', cmap=plt.cm.Set3)
    plt.xlim(ar_X.min(), ar_X.max())
    plt.ylim(ar_Y.min(), ar_Y.max())
    
    if save_idx != None:
        if not os.path.exists("gifs"):
            os.mkdir("gifs")
        plt.savefig("gifs/" + str(save_idx).zfill(5) + ".png")
    else:
        plt.show()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
visualize(X, Y, Z, model, device)

np.column_stack([X, Y, Z])자, 그럼 학습을 시작해볼게요.  
지난번과 같이 Adam Optimizer를 통해 학습을 진행해 보겠습니다.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
dataset = XORDataset()
dataloader = data.DataLoader(dataset, batch_size=512, shuffle=True)
model = LinearRegressionModel().to(device)

n_epoch = 200
losses = []
for epoch in range(n_epoch):
    for x, y in dataloader:
        x = x.to(device)
        y = y.to(device)
        
        # Initialize Optimizer
        optimizer.zero_grad()
        
        # Model Inference
        y_hat = model(x)
        
        # Get Loss
        loss = criterion(y_hat, y)
        
        # Logging
        losses.append(loss.detach().cpu().item())
        
        # Back Propagation
        loss.backward()
        
        # Weight Update
        optimizer.step()
    
    if epoch%10 == 0:
        print('EPOCH[{current}|{total}] loss: {loss:.2f}'.format(
            current=epoch+1,
            total=n_epoch,
            loss=losses[-1]
        ))

자, 학습이 완료되었습니다!  
학습이 되는동안 loss가 어떻게 떨어졌는지 Visualize해볼게요.

In [ ]:
plt.plot(losses)
plt.title("Loss")
plt.show()

Loss가 잘 안떨어지네요?   
그러면 모델이 어떻게 학습했는지 Visualize를 해볼게요.

In [ ]:
visualize(X, Y, Z, model, device=device)

자, 그러면 Model을 바꿔서, Multi Layer Perceptron을 적용시켜봅시다!

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(2, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [ ]:
model = MLP().to(device)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
dataset = XORDataset()
dataloader = data.DataLoader(dataset, batch_size=128, shuffle=True)

n_epoch = 200
losses = []
for epoch in range(n_epoch):
    for x, y in dataloader:
        x = x.to(device)
        y = y.to(device)
        
        # Initialize Optimizer
        optimizer.zero_grad()
        
        # Model Inference
        y_hat = model(x)
        
        # Get Loss
        loss = criterion(y_hat, y)
        
        # Logging
        losses.append(loss.detach().cpu().item())
        
        # Back Propagation
        loss.backward()
        
        # Weight Update
        optimizer.step()
    
    if epoch%10 == 0:
        print('EPOCH[{current}|{total}] loss: {loss:.2f}'.format(
            current=epoch+1,
            total=n_epoch,
            loss=losses[-1]
        ))

In [ ]:
plt.plot(losses)
plt.title("Loss")
plt.show()

In [ ]:
visualize(X, Y, Z, model, device=device)

## 수고하셨습니다